In [1]:
%matplotlib inline
import pylab as pl
import sympy
import sympy.physics.mechanics as mech
import control
sympy.init_printing(use_latex='mathjax')
mech.init_vprinting(use_latex='mathjax')

In [2]:
p_N, p_E, p_D, U, V, W, V_T, alpha, beta, P, Q, R, psi, theta, \
phi, t , m, J_x, J_y, J_z, J_xz = \
    sympy.symbols('p_N p_E p_D U V W V_T alpha beta P Q R psi theta phi t m J_x J_y J_z J_xz')
F_bx, F_by, F_bz, M_bx, M_by, M_bz = sympy.symbols('F_bx F_by F_bz M_bx M_by M_bz')
F_wx, F_wy, F_wz, M_wx, M_wy, M_wz = sympy.symbols('F_wx F_wy F_wz M_wx M_wy M_wz')

In [3]:
def bke(vect, frame_i, frame_b, t):
    return vect.diff(t, frame_b) + frame_b.ang_vel_in(frame_i).cross(vect)

#Define Frames and Rotation Rates

In [4]:
frame_i = mech.ReferenceFrame('i')
frame_b = frame_i.orientnew('b', 'Body', (psi(t), theta(t), phi(t)), '321')
frame_w = frame_b.orientnew('w', 'Body', (beta(t), -alpha(t), 0), '321')
omega_ib_b = P(t)*frame_b.x + Q(t)*frame_b.y + R(t)*frame_b.z
sol_euler_rates = sympy.solve((frame_b.ang_vel_in(frame_i) - omega_ib_b).to_matrix(frame_b),
            [xi(t).diff(t) for xi in [phi, theta, psi]])
frame_b.set_ang_vel(frame_i, omega_ib_b)
sol_euler_rates

⎧                                               Q⋅sin(φ) + R⋅cos(φ)           
⎨φ̇: P + Q⋅sin(φ)⋅tan(θ) + R⋅cos(φ)⋅tan(θ), ψ̇: ───────────────────, θ̇: Q⋅cos
⎩                                                      cos(θ)                 

              ⎫
(φ) - R⋅sin(φ)⎬
              ⎭

#Define Points and Velocities

In [5]:
p_i = p_N(t)*frame_i.x + p_E(t)*frame_i.y + p_D(t)*frame_i.z
point_o = mech.Point('o')
point_o.set_vel(frame_i, 0)

Define the Center of Mass with Velocity Described in the Body Frame

In [6]:
point_cm_b = point_o.locatenew('cm', p_i)
point_cm_b.set_vel(frame_b, 0)
V_i_b = U(t)*frame_b.x + V(t)*frame_b.y + W(t)*frame_b.z
point_cm_b.set_vel(frame_i, point_cm_b.pos_from(point_o).diff(t, frame_i))
sol_vel_body = sympy.solve((point_cm_b.vel(frame_i) - V_i_b).to_matrix(frame_i),
                      [xi(t).diff(t) for xi in [p_N, p_E, p_D]])
point_cm_b.set_vel(frame_i, V_i_b)
sol_vel_body

{p_̇D: -U⋅sin(θ) + V⋅sin(φ)⋅cos(θ) + W⋅cos(φ)⋅cos(θ), p_̇E: U⋅sin(ψ)⋅cos(θ) + 
V⋅sin(φ)⋅sin(ψ)⋅sin(θ) + V⋅cos(φ)⋅cos(ψ) - W⋅sin(φ)⋅cos(ψ) + W⋅sin(ψ)⋅sin(θ)⋅c
os(φ), p_̇N: U⋅cos(ψ)⋅cos(θ) + V⋅sin(φ)⋅sin(θ)⋅cos(ψ) - V⋅sin(ψ)⋅cos(φ) + W⋅si
n(φ)⋅sin(ψ) + W⋅sin(θ)⋅cos(φ)⋅cos(ψ)}

Define the Center of Mass with Velocity Described in the Wind Frame

In [7]:
point_cm_w = point_o.locatenew('cm', p_i)
point_cm_w.set_vel(frame_b, 0)
V_i_w = V_T(t)*frame_w.x
point_cm_w.set_vel(frame_i, point_cm_w.pos_from(point_o).diff(t, frame_i))
sol_vel_wind = sympy.solve((point_cm_w.vel(frame_i) - V_i_w).to_matrix(frame_i),
                      [xi(t).diff(t) for xi in [p_N, p_E, p_D]])
point_cm_w.set_vel(frame_i, V_i_w)
sol_vel_wind

{p_̇D: (sin(α)⋅cos(φ)⋅cos(θ) + sin(β)⋅sin(φ)⋅cos(α)⋅cos(θ) - sin(θ)⋅cos(α)⋅cos
(β))⋅V_T, p_̇E: (-sin(α)⋅sin(φ)⋅cos(ψ) + sin(α)⋅sin(ψ)⋅sin(θ)⋅cos(φ) + sin(β)⋅
sin(φ)⋅sin(ψ)⋅sin(θ)⋅cos(α) + sin(β)⋅cos(α)⋅cos(φ)⋅cos(ψ) + sin(ψ)⋅cos(α)⋅cos(
β)⋅cos(θ))⋅V_T, p_̇N: (sin(α)⋅sin(φ)⋅sin(ψ) + sin(α)⋅sin(θ)⋅cos(φ)⋅cos(ψ) + si
n(β)⋅sin(φ)⋅sin(θ)⋅cos(α)⋅cos(ψ) - sin(β)⋅sin(ψ)⋅cos(α)⋅cos(φ) + cos(α)⋅cos(β)
⋅cos(ψ)⋅cos(θ))⋅V_T}

#Rigid Body

In [8]:
J_cm_b = mech.inertia(frame_b, ixx=J_x, iyy=J_y, izz=J_z, izx=J_xz)
body_b = mech.RigidBody('body', point_cm_b, frame_b, m, (J_cm_b, point_cm_b))
body_w = mech.RigidBody('body', point_cm_w, frame_b, m, (J_cm_b, point_cm_w))

In [9]:
L_i_b = body_b.linear_momentum(frame_i)
F_b = F_bx(t)*frame_b.x + F_by(t)*frame_b.y + F_bz(t)*frame_b.z
sol_trans_body = sympy.solve((bke(L_i_b, frame_i, frame_b, t) - F_b).to_matrix(frame_b),
                        [xi(t).diff(t) for xi in [U, V, W]])
sol_trans_body

⎧                 F_bx                  F_by                   F_bz⎫
⎨U̇: -Q⋅W + R⋅V + ────, V̇: P⋅W - R⋅U + ────, Ẇ: -P⋅V + Q⋅U + ────⎬
⎩                  m                     m                      m  ⎭

In [10]:
L_i_w = body_w.linear_momentum(frame_i)
F_w = F_wx(t)*frame_w.x + F_wy(t)*frame_w.y + F_wz(t)*frame_w.z
sol_trans_wind = sympy.solve((bke(L_i_w, frame_i, frame_w, t) - F_w).to_matrix(frame_w),
                        [xi(t).diff(t) for xi in [V_T, alpha, beta]])
sol_trans_wind

⎧      F_wx                              F_wz                                 
⎨V_̇T: ────, α̇: -P⋅sin(β) + Q⋅cos(β) + ─────, β̇: P⋅cos(β)⋅tan(α) + Q⋅sin(β)⋅
⎩       m                               m⋅V_T                                 

                 F_wy    ⎫
tan(α) - R + ────────────⎬
             m⋅V_T⋅cos(α)⎭

In [11]:
H_cm_i_b = body_b.angular_momentum(point_cm_b, frame_i)
M_b = M_bx*frame_b.x + M_by*frame_b.y + M_bz*frame_b.z
sol_rot = sympy.solve((bke(H_cm_i_b, frame_i, frame_b, t) - M_b).to_matrix(frame_b),
                      [xi(t).diff(t) for xi in [P, Q, R]])
sol_rot

⎧                                                                             
⎪    -(J_xz⋅(Jₓ⋅P⋅Q + J_xz⋅Q⋅R - J_y⋅P⋅Q + M_bz) + J_z⋅(J_xz⋅P⋅Q - J_y⋅Q⋅R + J
⎨Ṗ: ─────────────────────────────────────────────────────────────────────────
⎪                                                       2                     
⎩                                          Jₓ⋅J_z - J_xz                      

                                      2         2                             
_z⋅Q⋅R - M_bx))       -Jₓ⋅P⋅R + J_xz⋅P  - J_xz⋅R  + J_z⋅P⋅R + M_by      Jₓ⋅(Jₓ
────────────────, Q̇: ────────────────────────────────────────────, Ṙ: ──────
                                          J_y                                 
                                                                              

                                                                              
⋅P⋅Q + J_xz⋅Q⋅R - J_y⋅P⋅Q + M_bz) + J_xz⋅(J_xz⋅P⋅Q - J_y⋅Q⋅R + J_z⋅Q⋅R - M_bx)
──────────────────────────────────────────────────